In [ ]:
%reload_kedro

In [ ]:
from crypto_thesis.data_domains.modeling.lstm import _build_lstm_timestamps_seq, lstm_model_predict
from pprint import pprint
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
import tensorflow as tf
import numpy as np
from keras.engine.sequential import Sequential
from keras.layers import LSTM, BatchNormalization, Dense
from keras.models import Sequential
from keras.regularizers import l2
from sklearn.model_selection import GridSearchCV
from keras.callbacks import EarlyStopping, ReduceLROnPlateau

## Base

In [ ]:
TARGET_COL = ["label"]
INDEX_COL = "window_nbr"

In [ ]:
mt_train_multic = catalog.load("master_table_train_multic")
mt_test_multic = catalog.load("master_table_test_multic")
seq_length = catalog.load("params:lstm_timestamp_seq_length")

## LSTM

In [ ]:
# set numpy seed
np.random.seed(0)
# set tensorflow seed
tf.random.set_seed(0)
SHUFFLE = False

EPOCHS = 50

tf.keras.utils.disable_interactive_logging()

In [ ]:
master_table_train = mt_train_multic.set_index(INDEX_COL)
X_train, y_train = master_table_train.drop(columns=TARGET_COL), master_table_train[TARGET_COL]

X_train_scaled_seq, y_train_scaled_seq = _build_lstm_timestamps_seq(X=X_train,
                                                                    y=y_train,
                                                                    seq_length=seq_length)

master_table_test = mt_test_multic.set_index(INDEX_COL)
X_test, y_test = master_table_test.drop(columns=TARGET_COL), master_table_test[TARGET_COL]

X_test_scaled_seq, y_test_scaled_seq = _build_lstm_timestamps_seq(X=X_test,
                                                                    y=y_test,
                                                                    seq_length=seq_length)

In [ ]:
def create_model_lstm(optimizer: str) -> Sequential:

    # parameters
    LAYERS = [20, 20, 20, 1] #[10, 10, 10, 1]                # number of units in hidden and output layers
    N = X_train_scaled_seq.shape[2]                 # number of features
    LAMBD = 0.005 #0.001                         # lambda in L2 regularizaion
    DP = 0.0 #0.0                             # dropout rate
    RDP = 0.0 #0.0                            # recurrent dropout rate

    # model
    model = Sequential()
    model.add(LSTM(
        input_shape=(seq_length, N),
        units=LAYERS[0],
        activation='tanh',
        recurrent_activation='hard_sigmoid',
        kernel_regularizer=l2(LAMBD),
        recurrent_regularizer=l2(LAMBD),
        dropout=DP,
        recurrent_dropout=RDP,
        return_sequences=True,
        return_state=False,
        stateful=False,
        unroll=False
                ))
    model.add(BatchNormalization())
    model.add(LSTM(
        units=LAYERS[1],
        activation='tanh',
        recurrent_activation='hard_sigmoid',
        kernel_regularizer=l2(LAMBD),
        recurrent_regularizer=l2(LAMBD),
        dropout=DP,
        recurrent_dropout=RDP,
        return_sequences=True,
        return_state=False,
        stateful=False,
        unroll=False
                ))
    model.add(BatchNormalization())
    model.add(LSTM(
        units=LAYERS[2],
        activation='tanh',
        recurrent_activation='hard_sigmoid',
        kernel_regularizer=l2(LAMBD),
        recurrent_regularizer=l2(LAMBD),
        dropout=DP,
        recurrent_dropout=RDP,
        return_sequences=False,
        return_state=False,
        stateful=False,
        unroll=False
                ))
    model.add(BatchNormalization())
    model.add(Dense(
        units=LAYERS[3],
        activation='sigmoid'))
    
    model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    
    return model

In [ ]:
lstm_model_params = {
    'batch_size': [4, 16],
    'optimizer': ['SGD', 'RMSprop', 'Adam']
    }
model = KerasClassifier(build_fn=create_model_lstm, epochs=EPOCHS, verbose=1)

grid = GridSearchCV(estimator=model, param_grid=lstm_model_params, n_jobs=1, cv=3)
grid_result = grid.fit(X_train_scaled_seq, y_train_scaled_seq)

best_params = grid_result.best_params_

print("*" * 100)
print()
print(f"Best accuracy of {grid_result.best_score_}") 
print()
print("Best parameters:")
pprint(best_params)

In [ ]:
model = create_model_lstm(optimizer=best_params["optimizer"])

M_TRAIN = X_train_scaled_seq.shape[0]           # number of training examples (2D)
M_TEST = X_test_scaled_seq.shape[0]             # number of test examples (2D),full=X_test.shape[0]
BATCH = best_params["batch_size"]                          # batch size

lr_decay = ReduceLROnPlateau(
            monitor='loss',
            patience=1,
            verbose=0,
            factor=0.5,
            min_lr=1e-8)

# Define Early Stopping:
early_stop = EarlyStopping(monitor='val_loss', min_delta=0,
                        patience=EPOCHS, verbose=1, mode='auto',
                        baseline=0, restore_best_weights=True)

train_history = model.fit(X_train_scaled_seq, y_train_scaled_seq,
                    epochs=EPOCHS,
                    batch_size=BATCH,
                    validation_split=0.0,
                    validation_data=(X_test_scaled_seq[:M_TEST], y_test_scaled_seq[:M_TEST]),
                    shuffle=SHUFFLE, verbose=0,
                    callbacks=[lr_decay, early_stop])

y_pred = lstm_model_predict(model=model, master_table_test=mt_test_multic, seq_length=seq_length)
print(y_pred["y_pred"].value_counts())